<a href="https://colab.research.google.com/github/osgeokr/kari-sdm/blob/main/240724_introduction_to_dynamic_world_part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee
import geemap

# Earth Engine 인증
ee.Authenticate()

# Earth Engine 초기화
ee.Initialize(project='ee-foss4g')

In [4]:
# 피처 컬렉션 객체 (행정구역)
countries = ee.FeatureCollection("FAO/GAUL/2015/level0")

# 피처 선택 ('ADM0_NAME' == '대한민국)
kr = countries.filter(ee.Filter.eq('ADM0_NAME', 'Republic of Korea'))

# 피처 컬렉션 출력 (대한민국)
m = geemap.Map(layout={'height':'400px', 'width':'800px'})
m.addLayer(kr, {}, "Republic of Korea")
m.centerObject(kr, 6)
m

Map(center=[36.34449219000809, 127.80177597242088], controls=(WidgetControl(options=['position', 'transparent_…

In [5]:
# 날짜 범위 설정
start_date = '2019-05-01'
end_date = '2023-07-31'

In [6]:
# Sentinel-2 이미지 컬렉션 필터링
s2 = (ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
      .filterDate(start_date, end_date)
      .filterBounds(kr.geometry())
      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 35)))

In [17]:
# Sentinel-2 이미지의 첫 번째 이미지 가져오기
s2_image = s2.first()

# 시각화 매개변수 설정
s2_vis_params = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 3000
}

m = geemap.Map(layout={'height':'400px', 'width':'800px'})
m.addLayer(s2_image, s2_vis_params, 'Sentinel-2 Image')
m.centerObject(s2_image, 9)
m

Map(center=[37.439844505445286, 124.75079548467365], controls=(WidgetControl(options=['position', 'transparent…

In [10]:
# 'system:index' 속성 가져오기
image_id = s2_image.get('system:index').getInfo()
print(image_id)

20190501T022559_20190501T022557_T51SXB


In [11]:
# Dynamic World 이미지 컬렉션 필터링
dw = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1").filter(
    ee.Filter.eq("system:index", image_id)
)

# 첫 번째 이미지 가져오기
dw_image = dw.first()

# 이미지 출력
print(dw_image.getInfo())

{'type': 'Image', 'bands': [{'id': 'water', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [10940, 10942], 'crs': 'EPSG:32651', 'crs_transform': [10, 0, 600190, 0, 10, 4090400]}, {'id': 'trees', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [10940, 10942], 'crs': 'EPSG:32651', 'crs_transform': [10, 0, 600190, 0, 10, 4090400]}, {'id': 'grass', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [10940, 10942], 'crs': 'EPSG:32651', 'crs_transform': [10, 0, 600190, 0, 10, 4090400]}, {'id': 'flooded_vegetation', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [10940, 10942], 'crs': 'EPSG:32651', 'crs_transform': [10, 0, 600190, 0, 10, 4090400]}, {'id': 'crops', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [10940, 10942], 'crs': 'EPSG:32651', 'crs_transform': [10, 0, 600190, 0, 10, 4090400]}, {'id': 'shrub_and_scrub', 'data_type': {'type': 'PixelType', 'precision': 

In [16]:
# Dynamic World 이미지의 'label' 밴드 선택
classification = dw_image.select('label')

# 시각화 매개변수 설정
dw_vis_params = {
    'min': 0,
    'max': 8,
    'palette': [
        '#419BDF', '#397D49', '#88B053', '#7A87C6', '#E49635',
        '#DFC35A', '#C4281B', '#A59B8F', '#B39FE1'
    ]
}

m = geemap.Map(layout={'height':'400px', 'width':'800px'})
m.addLayer(classification, dw_vis_params, 'Classified Image')
m.centerObject(classification, 9)
m

Map(center=[37.43984474093899, 124.7507960341314], controls=(WidgetControl(options=['position', 'transparent_b…

In [18]:
# 확률 밴드 목록
probability_bands = [
    'water', 'trees', 'grass', 'flooded_vegetation', 'crops',
    'shrub_and_scrub', 'built', 'bare', 'snow_and_ice'
]

# 확률 밴드 선택
probability_image = dw_image.select(probability_bands)

# 각 픽셀에서 가장 높은 확률 값을 가지는 이미지 생성
top1_probability = probability_image.reduce(ee.Reducer.max())

In [21]:
# 확률 값을 정수로 변환
top1_confidence = top1_probability.multiply(100).int()

# hillshade 계산
hillshade = ee.Terrain.hillshade(top1_confidence).divide(255)

In [22]:
# 지형 음영을 시각화할 매개변수 설정
dw_vis_params = {
    'min': 0,
    'max': 8,
    'palette': [
        '#419BDF', '#397D49', '#88B053', '#7A87C6', '#E49635',
        '#DFC35A', '#C4281B', '#A59B8F', '#B39FE1'
    ]
}

# 분류 이미지를 색상화
rgb_image = classification.visualize(**dw_vis_params).divide(255)

# 지형 음영을 색상화
probability_hillshade = rgb_image.multiply(hillshade)

# 지형 음영 시각화 매개변수 설정
hillshade_vis_params = {'min': 0, 'max': 0.8}

m = geemap.Map(layout={'height':'400px', 'width':'800px'})
m.addLayer(probability_hillshade, hillshade_vis_params, 'Probability Hillshade')
m.centerObject(probability_hillshade, 9)
m

Map(center=[37.4398494548925, 124.7507853444971], controls=(WidgetControl(options=['position', 'transparent_bg…

In [23]:
# 날짜 범위 설정
start_date = '2019-05-01'
end_date = '2023-07-31'

# Dynamic World 이미지 컬렉션 필터링
dw = (ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1')
      .filterDate(start_date, end_date)
      .filterBounds(kr.geometry()))

In [24]:
# Dynamic World 이미지 컬렉션의 'label' 밴드 선택
classification = dw.select('label')

# 'mode' 리듀서를 사용하여 합성 이미지 생성
dw_composite = classification.reduce(ee.Reducer.mode())

In [27]:
# 시각화 매개변수 설정
dw_vis_params = {
    'min': 0,
    'max': 8,
    'palette': [
        '#419BDF', '#397D49', '#88B053', '#7A87C6', '#E49635', '#DFC35A',
        '#C4281B', '#A59B8F', '#B39FE1'
    ]
}

# 합성 이미지를 지리적 범위로 자르기
dw_composite_clipped = dw_composite.clip(kr.geometry())

m = geemap.Map(layout={'height':'400px', 'width':'800px'})
m.addLayer(dw_composite_clipped, dw_vis_params, 'Classified Composite')
m.centerObject(dw_composite_clipped, 9)
m

Map(center=[36.34449219000809, 127.80177597242088], controls=(WidgetControl(options=['position', 'transparent_…

In [28]:
# 확률 밴드 목록
probability_bands = [
    'water', 'trees', 'grass', 'flooded_vegetation', 'crops',
    'shrub_and_scrub', 'built', 'bare', 'snow_and_ice'
]

# 확률 밴드 선택
probability_col = dw.select(probability_bands)

# 각 클래스의 시간 기간 동안의 평균 픽셀 확률로 이미지를 생성
mean_probability = probability_col.reduce(ee.Reducer.mean())

In [29]:
# 프로젝션 설정
projection = ee.Projection('EPSG:3857').atScale(10)

# 평균 확률 이미지에 기본 프로젝션 설정
mean_probability = mean_probability.setDefaultProjection(projection)

In [35]:
# Top-1 Probability Hillshade 생성
top1_probability = mean_probability.reduce(ee.Reducer.max())
top1_confidence = top1_probability.multiply(100).int()
hillshade = ee.Terrain.hillshade(top1_confidence).divide(255)

# dwComposite를 시각화하고, RGB 이미지로 변환
rgb_image = dw_composite.visualize(**dw_vis_params).divide(255)

# 지형 음영을 색상화
probability_hillshade = rgb_image.multiply(hillshade)

# 지형 음영 시각화 매개변수 설정
hillshade_vis_params = {'min': 0, 'max': 0.8}

# 지형 음영을 지리적 범위로 자르기
probability_hillshade_clipped = probability_hillshade.clip(kr.geometry())

m = geemap.Map(layout={'height':'400px', 'width':'800px'})
m.addLayer(probability_hillshade_clipped, hillshade_vis_params, 'Probability Hillshade')
m.centerObject(probability_hillshade_clipped, 9)
m

Map(center=[36.34449219000809, 127.80177597242088], controls=(WidgetControl(options=['position', 'transparent_…

In [37]:
# 이미지 내보내기
task = ee.batch.Export.image.toDrive(
    image=dw_composite_clipped,
    description='dw_composite_raw',
    region=kr.geometry(),
    scale=10,
    maxPixels=1e10
)

# 작업 시작
task.start()

print('Exporting image to Google Drive...')

Exporting image to Google Drive...


In [38]:
# Top-1 Probability Hillshade Composite 시각화
hillshade_composite = probability_hillshade.visualize(**hillshade_vis_params)

# 이미지 내보내기
task = ee.batch.Export.image.toDrive(
    image=hillshade_composite.clip(kr.geometry()),
    description='dw_composite_hillshade',
    region=kr.geometry(),
    scale=10,
    maxPixels=1e10
)

# 작업 시작
task.start()

print('Exporting image to Google Drive...')

Exporting image to Google Drive...
